In [23]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

# Veri seti boyutu
n_samples = 1000

# Veri seti özellikleri
property_types = ["oda", "daire", "villa"]
locations = ["şehir merkezi", "kırsal"]
pool_options = [0, 1]  # 0: Yok, 1: Var
wifi_options = [0, 1]  # 0: Yok, 1: Var
breakfast_options = [0, 1]  # 0: Yok, 1: Var

In [24]:
# Veri seti oluşturma
random.seed(42)
np.random.seed(42)

data = {
    "Ev Tipi": random.choices(property_types, k=n_samples),
    "Konum": random.choices(locations, k=n_samples),
    "Oda Sayısı": np.random.randint(1, 6, size=n_samples),
    "Kişi Sayısı": np.random.randint(1, 10, size=n_samples),
    "Havuz": random.choices(pool_options, k=n_samples),
    "WiFi": random.choices(wifi_options, k=n_samples),
    "Kahvaltı": random.choices(breakfast_options, k=n_samples),
    "Günlük Fiyat": np.random.randint(100, 1000, size=n_samples),
}

# DataFrame'e dönüştürme
df = pd.DataFrame(data)

# Veri setinin ilk 5 satırını ekrana yazdırma
print("Veri setinin ilk 5 satırı:")
print(df.head())

Veri setinin ilk 5 satırı:
  Ev Tipi          Konum  Oda Sayısı  Kişi Sayısı  Havuz  WiFi  Kahvaltı  \
0   daire  şehir merkezi           4            4      0     1         0   
1     oda         kırsal           5            4      1     1         0   
2     oda         kırsal           3            5      0     1         0   
3     oda         kırsal           5            8      0     0         1   
4   villa  şehir merkezi           5            5      0     0         1   

   Günlük Fiyat  
0           141  
1           777  
2           693  
3           528  
4           902  


In [25]:
# Kategorik verileri one-hot encoding ile sayısal hale getirme
if "Ev Tipi" in df.columns and "Konum" in df.columns:
    df = pd.get_dummies(df, columns=["Ev Tipi", "Konum"], drop_first=True)
else:
    print("Hata: Kategorik kolonlar bulunamadı!")

# Özellikler (X) ve hedef sütunu (Y) ayırma
if "Günlük Fiyat" in df.columns:
    X = df.drop("Günlük Fiyat", axis=1)
    y = df["Günlük Fiyat"]
else:
    print("Hata: Hedef kolon (Günlük Fiyat) bulunamadı!")
    X = pd.DataFrame()
    y = pd.Series()


In [27]:
# Eğitim ve test setlerine ayırma
if not X.empty and not y.empty:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Eğitim ve test setlerinin boyutlarını ekrana yazdırma
    print("\nEğitim ve test setlerinin boyutları:")
    print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
    print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")
    
    print("\nVeri ön işleme tamamlandı ve veriler kaydedildi!")
else:
    print("Hata: Eğitim ve test setleri oluşturulamadı!")


Eğitim ve test setlerinin boyutları:
X_train: (800, 8), X_test: (200, 8)
y_train: (800,), y_test: (200,)

Veri ön işleme tamamlandı ve veriler kaydedildi!


In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import joblib
import warnings

# Uyarıları gizle
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Veri setini yükleme
full_data = pd.read_csv("../data/veri.csv")

In [41]:
# Kategorik verileri one-hot encoding ile sayısal hale getirme
full_data = pd.get_dummies(full_data, columns=["Ev Tipi", "Konum"], drop_first=True)

# Özellikler ve hedef sütunları ayırma
X = full_data.drop("Günlük Fiyat", axis=1)
y = full_data["Günlük Fiyat"]

# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Algoritma listesi
algorithms = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "Support Vector Machine": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Neural Network (MLP)": MLPRegressor(max_iter=500)
}

In [43]:
# Modelleri eğit ve doğrulukları ölç
best_model = None
best_rmse = float("inf")
results = []

for name, model in algorithms.items():
    print(f"Model: {name}")
    try:
        # Modeli eğit
        model.fit(X_train, y_train)
        # Tahmin yap
        predictions = model.predict(X_test)
        # RMSE hesapla
        rmse = mean_squared_error(y_test, predictions, squared=False)
        results.append((name, rmse))
        print(f"RMSE: {rmse:.4f}\n")
        # En iyi modeli seç
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model
    except Exception as e:
        print(f"{name} modeli hata verdi: {e}\n")

# En iyi modeli kaydetme
if best_model is not None:
    joblib.dump(best_model, "../models/eniyi.joblib")
    best_model_name = type(best_model).__name__
    print(f"En iyi model kaydedildi: {best_model_name} ile RMSE: {best_rmse:.4f}")
else:
    best_model_name = "Hiçbiri"
    print("En iyi model bulunamadı ve kaydedilemedi.")

# Sonuçları göster
print("\nModel Performans Sonuçları:")
for name, rmse in results:
    print(f"{name}: RMSE = {rmse:.4f}")

print(f"\nSeçilen En İyi Model: {best_model_name} ile RMSE: {best_rmse:.4f}")

Model: Linear Regression
RMSE: 263.3888

Model: Ridge Regression
RMSE: 263.3883

Model: Lasso Regression
RMSE: 263.1790

Model: Decision Tree
RMSE: 324.3786

Model: Random Forest
RMSE: 295.8948

Model: Gradient Boosting
RMSE: 263.9975

Model: AdaBoost
RMSE: 263.1359

Model: Support Vector Machine
RMSE: 263.7186

Model: K-Neighbors Regressor
RMSE: 284.7767

Model: Neural Network (MLP)
RMSE: 263.2072

En iyi model kaydedildi: AdaBoostRegressor ile RMSE: 263.1359

Model Performans Sonuçları:
Linear Regression: RMSE = 263.3888
Ridge Regression: RMSE = 263.3883
Lasso Regression: RMSE = 263.1790
Decision Tree: RMSE = 324.3786
Random Forest: RMSE = 295.8948
Gradient Boosting: RMSE = 263.9975
AdaBoost: RMSE = 263.1359
Support Vector Machine: RMSE = 263.7186
K-Neighbors Regressor: RMSE = 284.7767
Neural Network (MLP): RMSE = 263.2072

Seçilen En İyi Model: AdaBoostRegressor ile RMSE: 263.1359
